In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf

In [2]:
data = np.load("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/final_data/Preprocessed_data.npy")
labels = np.load("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/final_data/labels.npy")
X_train, y_train = data[:int(len(data)*.8)], labels[:int(len(data)*.8)]
X_test, y_test = data[int(len(data)*.8):], labels[int(len(data)*.8):]
X_test = np.reshape(X_test, (-1, 292,144))
data= 0

In [2]:
data = np.load("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/final_data/Preprocessed_data.npy")
labels = np.load("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/final_data/labels.npy")

In [3]:
from sklearn.model_selection import train_test_split

hurricanes = []
new_labels = []
add_zero = False
index = 0
while index < len(data):
    if labels[index] ==1:
        hurricanes.append(data[index])
        new_labels.append(1)
    index+=1
    
index =0
count = 0
length = len(hurricanes)
while count < length:
    if labels[index] == 0:
        hurricanes.append(data[index])
        new_labels.append(0)
        count +=1
    index +=1
hurricanes = np.array(hurricanes)

X_train, X_test, y_train, y_test = train_test_split(hurricanes, new_labels, test_size=0.2, random_state=42)
X_test = np.reshape(X_test, (-1, 292, 144))

In [4]:
def next_batch(batch_size, index):
    X = X_train[index:(index+batch_size)]
    X = np.reshape(X, (-1, 292, 144))
    return X, y_train[index:(index+batch_size)]

In [5]:
tf.reset_default_graph()

n_steps = 292
n_inputs = 144
n_neurons = 100
n_outputs = 2
n_layers = 5

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

lstm_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons)
              for layer in range(n_layers)]
multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
top_layer_h_state = states[-1][1]
logits = tf.layers.dense(top_layer_h_state, n_outputs, name="softmax")
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [ ]:
n_epochs = 50
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0] // batch_size):
            X_batch, y_batch = next_batch(batch_size, iteration*batch_size)
            #X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

In [9]:
count=0
for i in y_train:
    if i ==1:
        count +=1
(count / len(y_train))*100

7.111515410958905

In [10]:
x,y = next_batch(10,0)

In [13]:
k = x[0]

In [18]:
k = k.reshape(42048)